In [1]:
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta

In [2]:
sample_input = pd.DataFrame(
    {'DATE': ['2015-01-01', '2015-02-01', '2015-02-03', '2015-02-10', '2015-02-14', '2015-03-15', '2015-05-01', '2015-10-01'],
    'CUSTOMER_ACCOUNT_ID': ['joe@signifyd.com', 'fraudster@fraud.com', 'fraudster@fraud.com', 'joe@signifyd.com', 'fraudster@fraud.com', 'joe@signifyd.com', 'joe@signifyd.com', 'joe@signifyd.com'],
    'EVENT_TYPE': ['PURCHASE', 'FRAUD_REPORT', 'FRAUD_REPORT', 'PURCHASE', 'PURCHASE', 'PURCHASE', 'PURCHASE', 'PURCHASE']}
)

In [3]:
sample_input

,DATE,CUSTOMER_ACCOUNT_ID,EVENT_TYPE
0,2015-01-01,joe@signifyd.com,PURCHASE
1,2015-02-01,fraudster@fraud.com,FRAUD_REPORT
2,2015-02-03,fraudster@fraud.com,FRAUD_REPORT
3,2015-02-10,joe@signifyd.com,PURCHASE
4,2015-02-14,fraudster@fraud.com,PURCHASE
5,2015-03-15,joe@signifyd.com,PURCHASE
6,2015-05-01,joe@signifyd.com,PURCHASE
7,2015-10-01,joe@signifyd.com,PURCHASE


In [4]:
# assume the input is a pandas data frame after loading raw data records (e.g. csv/excel)
def get_dict(input_records):
    dict_records = dict()
    for index, row in input_records.iterrows():
        if not dict_records.get(row['CUSTOMER_ACCOUNT_ID']):
            dict_records[row['CUSTOMER_ACCOUNT_ID']] = {datetime.strptime(row['DATE'],'%Y-%m-%d'): row['EVENT_TYPE']}
        else:
            dict_records[row['CUSTOMER_ACCOUNT_ID']][datetime.strptime(row['DATE'],'%Y-%m-%d')] = row['EVENT_TYPE']
    return dict_records

In [5]:
dict_records = get_dict(sample_input)
dict_records

{'joe@signifyd.com': {datetime.datetime(2015, 1, 1, 0, 0): 'PURCHASE',
  datetime.datetime(2015, 2, 10, 0, 0): 'PURCHASE',
  datetime.datetime(2015, 3, 15, 0, 0): 'PURCHASE',
  datetime.datetime(2015, 5, 1, 0, 0): 'PURCHASE',
  datetime.datetime(2015, 10, 1, 0, 0): 'PURCHASE'},
 'fraudster@fraud.com': {datetime.datetime(2015, 2, 1, 0, 0): 'FRAUD_REPORT',
  datetime.datetime(2015, 2, 3, 0, 0): 'FRAUD_REPORT',
  datetime.datetime(2015, 2, 14, 0, 0): 'PURCHASE'}}

In [6]:
def status_report(dict_records):
    for id in dict_records:
        customer_hist = list(dict_records[id].keys())
        fraud_no = 0
        unconfirmed_no = 0
        date_compare = []
        for index, event_date in enumerate(customer_hist):
            purchase_no = 0
            confirmed_no = 0
            if index == 0 and dict_records[id][event_date] == 'PURCHASE':
                dict_records[id][event_date] = 'NO_HISTORY'
                purchase_no = 1
                date_compare.append(event_date)
            if dict_records[id][event_date] == 'FRAUD_REPORT':
                fraud_no += 1
            if index != 0 and dict_records[id][event_date] == 'PURCHASE':
                purchase_no += 1
                confirmed_no = sum([1 if (event_date - i) > timedelta(90) else 0 for i in date_compare])
                if confirmed_no > 0 and purchase_no > 0 and fraud_no == 0:
                    dict_records[id][event_date] = f'GOOD_HISTORY:{confirmed_no}'
                if confirmed_no == 0 and purchase_no > 0 and fraud_no == 0:
                    unconfirmed_no += 1
                    dict_records[id][event_date] = f'UNCONFIRMED_HISTORY:{unconfirmed_no}'
                if fraud_no > 0:
                    dict_records[id][event_date] = f'FRAUD_HISTORY:{fraud_no}'
                date_compare.append(event_date)
    return dict_records

In [7]:
status_report = status_report(dict_records)
status_report

{'joe@signifyd.com': {datetime.datetime(2015, 1, 1, 0, 0): 'NO_HISTORY',
  datetime.datetime(2015, 2, 10, 0, 0): 'UNCONFIRMED_HISTORY:1',
  datetime.datetime(2015, 3, 15, 0, 0): 'UNCONFIRMED_HISTORY:2',
  datetime.datetime(2015, 5, 1, 0, 0): 'GOOD_HISTORY:1',
  datetime.datetime(2015, 10, 1, 0, 0): 'GOOD_HISTORY:4'},
 'fraudster@fraud.com': {datetime.datetime(2015, 2, 1, 0, 0): 'FRAUD_REPORT',
  datetime.datetime(2015, 2, 3, 0, 0): 'FRAUD_REPORT',
  datetime.datetime(2015, 2, 14, 0, 0): 'FRAUD_HISTORY:2'}}

In [8]:
def solution(status_report):
    output = pd.DataFrame(columns = ['DATE', 'CUSTOMER_ACCOUNT_ID', 'EVENT_TYPE'])
    for entry in status_report:
        for items in status_report[entry].items():
            output = output.append({'DATE':items[0], 'CUSTOMER_ACCOUNT_ID':entry, 'EVENT_TYPE':items[1]}, ignore_index=True)
    output.drop(output[output['EVENT_TYPE'] == 'PURCHASE'].index, inplace = True)
    output.drop(output[output['EVENT_TYPE'] == 'FRAUD_REPORT'].index, inplace = True)
    output = output.sort_values(by = 'DATE')
    output = output.reset_index(drop=True)
    return output

In [9]:
sample_output = solution(status_report)
sample_output

,DATE,CUSTOMER_ACCOUNT_ID,EVENT_TYPE
0,2015-01-01,joe@signifyd.com,NO_HISTORY
1,2015-02-10,joe@signifyd.com,UNCONFIRMED_HISTORY:1
2,2015-02-14,fraudster@fraud.com,FRAUD_HISTORY:2
3,2015-03-15,joe@signifyd.com,UNCONFIRMED_HISTORY:2
4,2015-05-01,joe@signifyd.com,GOOD_HISTORY:1
5,2015-10-01,joe@signifyd.com,GOOD_HISTORY:4
